# Options Arbitrage

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from black_scholes import call_value, put_value, call_delta, put_delta, call_vega, put_vega
%matplotlib inline

The code above is a set of instructions that tell the computer what to do. First, it tells the computer to use some tools that help with working with data (pandas), doing math (numpy), and making graphs (matplotlib). Then, it tells the computer to use a special tool called "Black-Scholes" to do some calculations related to buying and selling stocks. Finally, it says to make sure we can see any graphs created within our notebook (the %matplotlib inline part). Basically, this code sets up the tools and methods we need to do some stock trading and analysis in Python.

In [2]:
def read_data(filename):
    df = pd.read_csv(filename, index_col=0)

    time_to_expiry = df.filter(like='TimeToExpiry')

    stock = df.filter(like='Stock')
    stock.columns = [stock.columns.str[-5:], stock.columns.str[:-6]]

    options = pd.concat((df.filter(like='-P'), df.filter(like='-C')), axis=1)
    options.columns = [options.columns.str[-3:], options.columns.str[:-4]]

    market_data = pd.concat((stock, options), axis=1)

    return time_to_expiry, market_data

This code takes a file with data in it and reads it using a function called "read_data". Then, it takes the data from that file and organizes it into a table using a library called "pandas". It separates the data into different categories, such as time to expiry and different types of stocks and options. Finally, it puts all of this data together in a new table called "market_data" and returns it to be used later.

In [3]:
# Read the market data
filename = 'Options Arbitrage.csv'
time_to_expiry, market_data = read_data(filename)

This python code reads a file called "Options Arbitrage.csv" that contains market data. It then uses a function called "read_data" to process the data and store it in two variables: "time_to_expiry" and "market_data". This information can then be used for further calculations or analysis related to options trading.

In [4]:
# Get a list of all instrument names including the stock, and of the options only
instrument_names = list(market_data.columns.get_level_values(0).unique())
print(instrument_names)

option_names = instrument_names[1:]
print(option_names)

['Stock', 'P60', 'P70', 'P80', 'C60', 'C70', 'C80']
['P60', 'P70', 'P80', 'C60', 'C70', 'C80']


This code gets a list of all the names of different things called "instruments" that make up a large group. This list includes some things called "stocks" and also some things called "options". The code makes this list by using a specific function for columns in something called "market data". The code then prints the list for you to see. Lastly, the code creates another list just for the "options" included in the first list and prints that list as well.

In [5]:
# Add time_to_expiry to market_data
market_data['TTE'] = time_to_expiry['TimeToExpiry']

# Store timestamp in variable to prevent
# errors with multiplications and such
timestamp = market_data.index

# Set the Time to Expiry as Index
market_data = market_data.set_index('TTE')

This code adds a new column called 'TTE' to some information called 'market_data'. This new column shows the length of time until something expires. To make sure everything runs smoothly, the code assigns the expiry time to a variable called 'timestamp' and sets it as the index (or main way to organize) for the data. This will help with other calculations and operations later on.

In [6]:
# Create Empty Dictionaries
short_call_values = {}
long_call_values = {}
long_put_values = {}
short_put_values = {}
short_call_deltas = {}
long_call_deltas = {}
long_put_deltas = {}
short_put_deltas = {}
option_values = {}
option_deltas = {}

# Set known attributes
r = 0
sigma = 0.20

# Forloop to create new columns with Call/Put names
for option in option_names:
    # Retrieve K from the Option
    K = int(option[-2:])

    if 'C' in option:
        short_call_values[option] = []
        long_call_values[option] = []
        short_call_deltas[option] = []
        long_call_deltas[option] = []

        # Forloop to calculate short/long call values and deltas
        for time, stock_value in market_data.iterrows():
            short_call_values[option].append(call_value(
                stock_value['Stock', 'AskPrice'], K, time, r, sigma))
            long_call_values[option].append(call_value(
                stock_value['Stock', 'BidPrice'], K, time, r, sigma))
            long_call_deltas[option].append(call_delta(
                stock_value['Stock', 'BidPrice'], K, time, r, sigma))
            short_call_deltas[option].append(-call_delta(
                stock_value['Stock', 'AskPrice'], K, time, r, sigma))

        option_values['Short Call', option] = short_call_values[option]
        option_values['Long Call', option] = long_call_values[option]
        option_deltas['Short Call', option] = short_call_deltas[option]
        option_deltas['Long Call', option] = long_call_deltas[option]

    if 'P' in option:
        long_put_values[option] = []
        short_put_values[option] = []
        long_put_deltas[option] = []
        short_put_deltas[option] = []

        # Forloop to calculate short/long put values and deltas
        for time, stock_value in market_data.iterrows():
            long_put_values[option].append(
                put_value(stock_value['Stock', 'AskPrice'], K, time, r, sigma))
            short_put_values[option].append(
                put_value(stock_value['Stock', 'BidPrice'], K, time, r, sigma))
            long_put_deltas[option].append(
                put_delta(stock_value['Stock', 'AskPrice'], K, time, r, sigma))
            short_put_deltas[option].append(-put_delta(
                stock_value['Stock', 'BidPrice'], K, time, r, sigma))

        option_values['Long Put', option] = long_put_values[option]
        option_values['Short Put', option] = short_put_values[option]
        option_deltas['Long Put', option] = long_put_deltas[option]
        option_deltas['Short Put', option] = short_put_deltas[option]

This code is creating empty dictionaries to store values and deltas for different types of stock options. It then sets two known attributes, the risk-free rate "r" and the volatility "sigma". Next, it uses a loop to go through all the option names and retrieve the strike price "K" from the option name. If the option is a call, it calculates the short and long call values and deltas using the stock value data. If the option is a put, it calculates the short and long put values and deltas. It then adds these values to the corresponding dictionaries. Finally, the code adds the calculated values and deltas to the overall option values and deltas dictionaries.

In [7]:
# Create DataFrames with index market_data
option_values = pd.DataFrame(option_values, index=market_data.index)
option_deltas = pd.DataFrame(option_deltas, index=market_data.index)

# Sort the DataFrames
option_values = option_values.reindex(sorted(option_values.columns), axis=1)
option_deltas = option_deltas.reindex(sorted(option_deltas.columns), axis=1)

# Rounding
option_values = round(option_values, 2)

# Show DataFrames
print('Option Values')
display(option_values.head())
print('Option Deltas')
display(option_deltas.head())

Option Values


Long Call             Long Put              Short Call              \
               C60   C70   C80      P60   P70    P80        C60   C70   C80   
TTE                                                                           
0.911625     12.04  5.71  2.24     1.31  4.92  11.39      12.21  5.82  2.29   
0.911615     12.08  5.74  2.25     1.29  4.88  11.32      12.29  5.88  2.32   
0.911606     12.17  5.79  2.28     1.28  4.86  11.29      12.33  5.91  2.34   
0.911596     12.12  5.76  2.27     1.28  4.86  11.29      12.33  5.91  2.34   
0.911587     12.08  5.74  2.25     1.29  4.88  11.32      12.29  5.88  2.32   

         Short Put               
               P60   P70    P80  
TTE                              
0.911625      1.34  5.01  11.54  
0.911615      1.33  4.99  11.50  
0.911606      1.32  4.94  11.43  
0.911596      1.32  4.96  11.47  
0.911587      1.33  4.99  11.50

Option Deltas


Long Call                      Long Put                      \
               C60       C70       C80       P60       P70       P80   
TTE                                                                    
0.911625  0.830169  0.558665  0.290582 -0.166116 -0.435503 -0.704329   
0.911615  0.831105  0.560126  0.291851 -0.164281 -0.432599 -0.701776   
0.911606  0.832963  0.563041  0.294394 -0.163369 -0.431150 -0.700497   
0.911596  0.832038  0.561584  0.293120 -0.163368 -0.431150 -0.700499   
0.911587  0.831108  0.560126  0.291847 -0.164278 -0.432599 -0.701779   

         Short Call                     Short Put                      
                C60       C70       C80       P60       P70       P80  
TTE                                                                    
0.911625  -0.833884 -0.564497 -0.295671  0.169831  0.441335  0.709418  
0.911615  -0.835719 -0.567401 -0.298224  0.168895  0.439874  0.708149  
0.911606  -0.836631 -0.568850 -0.299503  0.167037  0.436959  0.705606  
0.911596  -0.836632 -0.568850 -0.299501  0.167962  0.438416  0.706880  
0.911587  -0.835722 -0.567401 -0.298221  0.168892  0.439874  0.708153

This code is creating two data tables called DataFrames. The first one is called "market_data" and the other two are called "option_values" and "option_deltas". These data tables will contain a list of values and will be organized in rows and columns. The code then sorts the columns in alphabetical order and rounds the values to two decimal places. Finally, it prints and displays the data tables for us to see.

In [8]:
# Create Columns for Black Scholes Value in the Data Set
# This is used for later calculations (algorithm and such)

for option in option_names:
    if "C" in option:
        market_data[option,
                    'Expected AskPrice'] = option_values['Short Call', option]
        market_data[option,
                    'Expected BidPrice'] = option_values['Long Call', option]
        market_data[option,
                    'Delta Short'] = option_deltas['Short Call', option].values
        market_data[option,
                    'Delta Long'] = option_deltas['Long Call', option].values

    elif "P" in option:
        market_data[option,
                    'Expected AskPrice'] = option_values['Short Put', option]
        market_data[option,
                    'Expected BidPrice'] = option_values['Long Put', option]
        market_data[option,
                    'Delta Short'] = option_deltas['Short Put', option].values
        market_data[option,
                    'Delta Long'] = option_deltas['Long Put', option].values

# Sort Columns
market_data = market_data.reindex(sorted(market_data.columns), axis=1)

This code creates new columns in a dataset for the Black Scholes Value, which is a mathematical model used in finance to determine the price of options. The code does this by looping through all the different options in the dataset and checking if each one is a call option (the right to buy a certain asset at a specific price) or a put option (the right to sell a certain asset at a specific price). Depending on what type of option it is, the code uses different calculations to determine the expected ask and bid prices (the price at which someone is willing to sell or buy the option, respectively) and the delta values (a measure of how the option price changes in relation to the underlying asset's price). Finally, the code reorders the columns in the dataset to make sure they are in a consistent and organized manner.

In [9]:
def option_opportunities(option):
    '''
    This function gives arbitrage opportunities based on whether the price
    of the option is too high or too low. The results are used to 'eyeball' 
    if our final results match what this function displays. This works for 
    all Calls and Puts.
    '''
    if "C" in option:
        expected1 = market_data[option][(market_data[option, 'BidPrice'] - market_data[option,
                                                                                       'Expected AskPrice']) >= 0.10].drop('Expected BidPrice', axis=1)
        expected2 = market_data[option][(market_data[option, 'Expected BidPrice'] -
                                         market_data[option, 'AskPrice']) >= 0.10].drop('Expected AskPrice', axis=1)

    elif "P" in option:
        expected1 = market_data[option][(market_data[option, 'BidPrice'] - market_data[option,
                                                                                       'Expected AskPrice']) >= 0.10].drop('Expected BidPrice', axis=1)
        expected2 = market_data[option][(market_data[option, 'Expected BidPrice'] -
                                         market_data[option, 'AskPrice']) >= 0.10].drop('Expected AskPrice', axis=1)

    print('BidPrice is at least 0.10 higher than Expected AskPrice for Option ' + option)
    display(expected1)
    print('AskPrice is at least 0.10 lower than Expected BidPrice for Option ' + option)
    display(expected2)
    print('The amount of trades are', len(expected1) + len(expected2))

option_opportunities('C80')

BidPrice is at least 0.10 higher than Expected AskPrice for Option C80


AskPrice  AskVolume  BidPrice  BidVolume  Delta Long  Delta Short  \
TTE                                                                           
0.910616      2.47       22.0      2.40      161.0    0.287906    -0.294257   
0.908790      2.49       19.0      2.41       19.0    0.292730    -0.296557   
0.863975      2.34       21.0      2.27       15.0    0.282434    -0.288892   
0.846851      2.29       14.0      2.22       18.0    0.281149    -0.285046   

          Expected AskPrice  
TTE                          
0.910616               2.28  
0.908790               2.30  
0.863975               2.17  
0.846851               2.11

AskPrice is at least 0.10 lower than Expected BidPrice for Option C80


AskPrice  AskVolume  BidPrice  BidVolume  Delta Long  Delta Short  \
TTE                                                                           
0.906355      2.03       17.0      1.96       19.0    0.282249    -0.288574   
0.890925      1.70      122.0      1.63       21.0    0.258776    -0.262483   
0.869368      1.31       86.0      1.25       19.0    0.218205    -0.222875   
0.847146      1.96      143.0      1.88       19.0    0.279899    -0.285090   
0.831088      2.43       14.0      2.35       16.0    0.324083    -0.329540   
0.755689      3.13       22.0      3.05       21.0    0.388099    -0.395452   
0.754014      3.09       95.0      3.00       18.0    0.386434    -0.392318   

          Expected BidPrice  
TTE                          
0.906355               2.14  
0.890925               1.87  
0.869368               1.45  
0.847146               2.06  
0.831088               2.54  
0.755689               3.23  
0.754014               3.21

The amount of trades are 11


This code is a function that helps us identify if there are any opportunities to make money in the stock market using options. It takes in an option (a type of investment) as an input and checks if the price of that option is too high or too low. It does this by comparing the market data (information about the stock market) for that option with a specific amount (0.10).   If the input option is a call option ("C" in option), the function looks at the data for that option and identifies if the difference between the bid price (the amount someone is willing to pay for the option) and expected ask price (the amount someone is willing to sell the option for) is at least 0.10. If it is, then the function displays the result and shows the trade opportunities where you can buy the option for a lower price and potentially sell it for a higher price.   If the option is a put option ("P" in option), the function does the same comparison but for a put option instead. After finding the opportunities for both buying and selling options, the function prints out the total number of trades available.   To use this function, we need to give it an option as an input, for example, "C80". The function will then run and show us any potential opportunities for buying and selling this particular option.

In [10]:
# Create a Dictionary with Timestamp and Time to Expiry
# Index of market_data was changed earlier to time to expiry
trades = {('Timestamp', ''): timestamp,
          ('Time to Expiry', ''): market_data.index}

# Forloop that adds columns for the Call/Put Positions and Deltas
# Global function is a changing variable name based on the option
# For option C60 it will create a variable named positions_call_C60
for option in option_names:

    if 'C' in option:
        trades['Call Position', option] = []
        trades['Call Delta', option] = []
        globals()['positions_call_' + option] = 0

    if 'P' in option:
        trades['Put Position', option] = []
        trades['Put Delta', option] = []
        globals()['positions_put_' + option] = 0

This code is used to create a dictionary in Python. The dictionary will have two key-value pairs: Timestamp and Time to Expiry. These values will be used to store the timestamp and the time to expiry of trades in a stock market. The code also contains a for loop that adds columns for the Call/Put positions and deltas. These columns will be used to store the positions and deltas of different options in the stock market. A global function is used to change the name of the variable based on the option. For example, for option C60, the variable name will be "positions_call_C60". It also checks if the option is a Call or Put option, and if it is a Call option, it creates a variable for the Call position and delta. Similarly, for Put options, it creates variables for the Put positions and deltas. This code will be helpful for organizing and analyzing data related to trades in the stock market.

In [11]:
# Forloop over the rows of market_data
for time, data in market_data.iterrows():

    max_delta = min(data['Stock', 'AskVolume'], data['Stock', 'BidVolume'])

    # Forloop over the option_names with conditions
    # if-statements if Call or Put + if Short/Long in Call or Put
    for option in option_names:

        if 'C' in option:

            # Short Call
            if (data[option, 'BidPrice'] - data[option, 'Expected AskPrice']) >= 0.10:
                short_call_volume = data[option, 'BidVolume']
                long_call_volume = 0

            # Long Call
            elif (data[option, 'Expected BidPrice'] - data[option, 'AskPrice']) >= 0.10:
                long_call_volume = data[option, 'AskVolume']
                short_call_volume = 0

            else:
                long_call_volume = short_call_volume = 0

            call_trade = long_call_volume - short_call_volume

            # Define variable, as set earlier. Note the first position is set to zero otherwise
            # One would get an error here since the variable is then not yet defined.
            globals()['positions_call_' + option] = call_trade + \
                globals()['positions_call_' + option]

            # Add Positions (cumulative)
            trades['Call Position', option].append(
                globals()['positions_call_' + option])

            if globals()['positions_call_' + option] >= 0:
                long_call_delta = data[option, 'Delta Long']
                short_call_delta = 0

            elif globals()['positions_call_' + option] < 0:
                short_call_delta = data[option, 'Delta Short']
                long_call_delta = 0

            # Add Deltas (cumulative)
            trades['Call Delta', option].append(
                abs(globals()['positions_call_' + option]) * (long_call_delta + short_call_delta))

        if 'P' in option:

            # Short Put
            if (data[option, 'BidPrice'] - data[option, 'Expected AskPrice']) >= 0.10:
                short_put_volume = data[option, 'BidVolume']
                long_put_volume = 0

            # Long Put
            elif (data[option, 'Expected BidPrice'] - data[option, 'AskPrice']) >= 0.10:
                long_put_volume = data[option, 'AskVolume']
                short_put_volume = 0

            else:
                long_put_volume = short_put_volume = 0

            put_trade = long_put_volume - short_put_volume

            globals()['positions_put_' + option] = put_trade + \
                globals()['positions_put_' + option]

            trades['Put Position', option].append(
                globals()['positions_put_' + option])

            if globals()['positions_put_' + option] >= 0:
                long_put_delta = data[option, 'Delta Long']
                short_put_delta = 0

            elif globals()['positions_put_' + option] < 0:
                short_put_delta = data[option, 'Delta Short']
                long_put_delta = 0

            trades['Put Delta', option].append(
                abs(globals()['positions_put_' + option]) * (long_put_delta + short_put_delta))

This code is analyzing stock market data and making trades based on certain conditions for options. It goes through the data row by row and determines the maximum difference between the bid and ask volumes for a specific stock. Then, it goes through a list of option names and checks if they are "call" or "put" options, as well as if they are "short" or "long". Depending on these conditions, it will either trade or not for that option.  The code then calculates the trade volume for each option based on the bid and ask prices. It also calculates the delta, which is a measure of how much the option price will change for a change in the underlying stock price. The position and delta for each option are then added to a list of cumulative trades and deltas.  Finally, the code repeats this process for put options and adds the positions and deltas for those to the list as well. Overall, the code is used to make trades and track changes in position and delta for different options based on market data.

In [12]:
# Create DataFrame with Index Timestamp
trades = pd.DataFrame(trades).set_index('Timestamp')

# Sort Columns
trades = trades.reindex(sorted(trades.columns), axis=1)

# Calculate Total Option Delta (based on sorted columns)
trades['Total Option Delta', ''] = np.sum(
    trades['Call Delta'], axis=1) + np.sum(trades['Put Delta'], axis=1)

# Calculate Cumulative Stock Position (floored if positive, ceiled if negative)
trades['Stock Position', 'Stock'] = -np.where(trades['Total Option Delta', ''] >= 0, np.floor(
    trades['Total Option Delta', '']), np.ceil(trades['Total Option Delta', '']))

# Calculate remaining option delta (that remains unhedged)
# This delta is included in the Total Option Delta again which ensures
# It always remains below zero
trades['Remaining Option Delta', ''] = trades['Total Option Delta',
                                              ''] + trades['Stock Position', 'Stock']

# Show DataFrame
trades.tail()

Call Delta                         Call Position         \
                            C60         C70         C80           C60    C70   
Timestamp                                                                      
2018-02-28 23:35:00  281.812848  375.708432  117.148014         305.0  532.0   
2018-02-28 23:40:00  281.646451  375.009738  116.724180         305.0  532.0   
2018-02-28 23:45:00  281.646800  375.010163  116.723895         305.0  532.0   
2018-02-28 23:50:00  281.647150  375.010589  116.723609         305.0  532.0   
2018-02-28 23:55:00  281.479734  374.309980  116.299862         305.0  532.0   

                           Put Delta                        Put Position  \
                       C80       P60        P70         P80          P60   
Timestamp                                                                  
2018-02-28 23:35:00  286.0 -7.017632 -10.388177  247.964456         95.0   
2018-02-28 23:40:00  286.0 -7.017525 -10.388147  248.586868         95.0   
2018-02-28 23:45:00  286.0 -7.017417 -10.388117  248.587287         95.0   
2018-02-28 23:50:00  286.0 -7.017310 -10.388087  248.587707         95.0   
2018-02-28 23:55:00  286.0 -7.118870 -10.481772  249.209993         95.0   

                                 Time to Expiry Total Option Delta  \
                      P70    P80                                     
Timestamp                                                            
2018-02-28 23:35:00  36.0 -420.0       0.750038        1005.227940   
2018-02-28 23:40:00  36.0 -420.0       0.750029        1004.561566   
2018-02-28 23:45:00  36.0 -420.0       0.750019        1004.562612   
2018-02-28 23:50:00  36.0 -420.0       0.750010        1004.563657   
2018-02-28 23:55:00  36.0 -420.0       0.750000        1003.698927   

                    Stock Position Remaining Option Delta  
                             Stock                         
Timestamp                                                  
2018-02-28 23:35:00        -1005.0               0.227940  
2018-02-28 23:40:00        -1004.0               0.561566  
2018-02-28 23:45:00        -1004.0               0.562612  
2018-02-28 23:50:00        -1004.0               0.563657  
2018-02-28 23:55:00        -1003.0               0.698927

This code creates a data structure called a DataFrame that contains information about trades. The first line sets the index of the DataFrame to be the timestamp of each trade. The next line sorts the columns of the DataFrame. The next few lines perform calculations on the data, specifically the total value of stock options and the position of the stock. The last line displays the resulting DataFrame. Essentially, this code is used for organizing and analyzing data related to trades.

In [13]:
# Create trades_diff dataframe that gives all actual trades (not positions)
# Also drop columns that are not required for PnL calculations and such
trades_diff = trades.diff()[1:].drop(
    ['Call Delta', 'Put Delta', 'Time to Expiry', 'Total Option Delta', 'Remaining Option Delta'], axis=1)

# Drop the 'Call Position','Put Position' and 'Stock Position' top level
# Makes forlooping easier
trades_diff.columns = trades_diff.columns.droplevel(level=0)

# Since positions are not neccesarily zero at the last timestamp, final positions are calculated to be able to valuate these
final_positions = trades[-1:].drop(['Call Delta', 'Put Delta', 'Time to Expiry',
                                    'Total Option Delta', 'Remaining Option Delta'], axis=1)

final_positions.columns = final_positions.columns.droplevel(level=0)

# Show DataFrames
print('Actual Trades/Volumes')
display(trades_diff.head())

print("Final Positions that we currently 'own'")
display(final_positions)

Actual Trades/Volumes


C60  C70  C80  P60  P70  P80  Stock
Timestamp                                               
2018-01-01 00:10:00  0.0  0.0  0.0  0.0  0.0  0.0    0.0
2018-01-01 00:15:00  0.0  0.0  0.0  0.0  0.0  0.0    0.0
2018-01-01 00:20:00  0.0  0.0  0.0  0.0  0.0  0.0    0.0
2018-01-01 00:25:00  0.0  0.0  0.0  0.0  0.0  0.0    0.0
2018-01-01 00:30:00  0.0  0.0  0.0  0.0  0.0  0.0    0.0

Final Positions that we currently 'own'


C60    C70    C80   P60   P70    P80   Stock
Timestamp                                                          
2018-02-28 23:55:00  305.0  532.0  286.0  95.0  36.0 -420.0 -1003.0

This code is used to create a dataframe (a table of data) that contains information about actual trades that have taken place. It first takes a larger dataframe called "trades" and calculates the changes in the data (difference) between each row by using the .diff() function. It then removes some unnecessary columns from this new dataframe, like information about the options and their expiry. Next, it gets rid of some other columns and changes the names of the columns so that it's easier to use. Lastly, it creates a new dataframe called "final_positions" that shows the current positions that we own based on the last row of data in the "trades" dataframe. This is done because the positions may not be zero at the last timestamp and we need to be able to value these positions. The code then prints out the two dataframes so that we can see the information in an organized way.

In [14]:
# Including Timestamp again to match trades_diff index
market_data['Timestamp'] = timestamp
market_data = market_data.set_index('Timestamp')

This code is written in the Python programming language. It is part of a larger program that is used to analyze and keep track of stock market data.  The first line of code is adding a new column called 'Timestamp' to the existing 'market_data' table. This column contains a series of time stamps, which are essentially labels used to keep track of when specific events or changes occurred.  The second line of code assigns the value of the 'timestamp' variable to the newly added 'Timestamp' column. This means that for each row in the 'market_data' table, the value of the 'timestamp' variable will be recorded in the 'Timestamp' column.  The third line of code changes the index of the 'market_data' table to be the 'Timestamp' column. This means that instead of being indexed by numbers, the rows in the table will now be indexed by the values in the 'Timestamp' column.  This process of adding a timestamp and changing the index is important because it allows the program to keep track of when data was collected and make it easier to analyze and compare changes over time. Think of it like a timeline for the market data, with each event or change labeled and organized in order.

In [15]:
# Create Dataframe with index market_data (timestamp)
cashflow_dataframe = pd.DataFrame(index=market_data.index[1:])

# Forloop on all instruments (including stock) to calculate PnL
for instrument in instrument_names:

    Instrument_AskPrice = market_data[instrument, 'AskPrice'][1:]
    Instrument_BidPrice = market_data[instrument, 'BidPrice'][1:]

    cashflow_dataframe[instrument] = np.where(trades_diff[instrument] >= 0,
                                              trades_diff[instrument] * -
                                              Instrument_AskPrice,
                                              trades_diff[instrument] * -Instrument_BidPrice)

# Show DataFrame & PnL
display(cashflow_dataframe.head())
total_cashflow = cashflow_dataframe.sum().sum()

print('The total Cashflow is: €', round(total_cashflow, 2))

Stock  P60  P70  P80  C60  C70  C80
Timestamp                                               
2018-01-01 00:10:00   -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0
2018-01-01 00:15:00   -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0
2018-01-01 00:20:00   -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0
2018-01-01 00:25:00   -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0
2018-01-01 00:30:00   -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0

The total Cashflow is: € 68985.54


This code creates a table (called a dataframe) that keeps track of the prices of different instruments (like stocks). It then uses a loop to calculate the profit or loss (PnL) for each instrument by looking at the difference between the amount of shares bought and sold and multiplying it by the price of each share. It displays this info in a table, along with the total profit or loss for all the instruments.

In [16]:
# Cumulative Cashflows
cashflow_cumulative = {}

for column in cashflow_dataframe.columns:

    cashflow_cumulative[column] = cashflow_dataframe[column].cumsum()

cashflow_cumulative = pd.DataFrame(cashflow_cumulative)

# Show Cumulative Cashflow
display(cashflow_cumulative.tail())

# Checking for Match
print('This number should match the above number: €',
      round(cashflow_cumulative[-1:].sum().sum(), 2))

Stock     P60     P70      P80      C60      C70  \
Timestamp                                                                  
2018-02-28 23:35:00  71187.00 -116.36  762.03  4326.29 -3141.69 -3354.85   
2018-02-28 23:40:00  71111.05 -116.36  762.03  4326.29 -3141.69 -3354.85   
2018-02-28 23:45:00  71111.05 -116.36  762.03  4326.29 -3141.69 -3354.85   
2018-02-28 23:50:00  71111.05 -116.36  762.03  4326.29 -3141.69 -3354.85   
2018-02-28 23:55:00  71035.20 -116.36  762.03  4326.29 -3141.69 -3354.85   

                        C80  
Timestamp                    
2018-02-28 23:35:00 -525.08  
2018-02-28 23:40:00 -525.08  
2018-02-28 23:45:00 -525.08  
2018-02-28 23:50:00 -525.08  
2018-02-28 23:55:00 -525.08

This number should match the above number: € 68985.54


This python code is trying to calculate the cumulative cashflows of a dataframe. A dataframe is a way to store and organize data. So first, it creates a dictionary called "cashflow_cumulative". A dictionary is like a list, but instead of just having numbers in it, it can have words too. Then, it loops through each of the columns in the dataframe (the different categories of data) and for each column, it calculates the cumulative sum (which means adding up all the numbers in the column so far). The results are then added to the dictionary. After that, the dictionary is turned into a new dataframe, and this one is printed out so you can see the final cumulative cashflow numbers. Finally, it does a quick check to make sure the final number in the dataframe (the last row and last column) matches the number you just printed out. It's like when you're doing a math problem and you check your answer to make sure it's right.

In [17]:
# Create a new dataframe of trades with most columns dropped
trades_minimal = trades.drop(['Call Delta', 'Put Delta', 'Time to Expiry', 'Total Option Delta',
                              'Remaining Option Delta'], axis=1)

trades_minimal.columns = trades_minimal.columns.droplevel(level=0)

# Create Dataframe with market_data as index
valuation_dataframe = pd.DataFrame(index=market_data.index)

# Forloop to calculate valuations on every timestamp
for instrument in instrument_names:

    if 'C' in instrument:

        Instrument_AskPrice = market_data[instrument, 'AskPrice']
        Instrument_BidPrice = market_data[instrument, 'BidPrice']

        valuation_dataframe[instrument] = np.where(trades_minimal[instrument] > 0,
                                                   trades_minimal[instrument] *
                                                   Instrument_BidPrice,
                                                   trades_minimal[instrument] * Instrument_AskPrice)

    if 'P' in instrument:

        Instrument_AskPrice = market_data[instrument, 'AskPrice']
        Instrument_BidPrice = market_data[instrument, 'BidPrice']

        valuation_dataframe[instrument] = np.where(trades_minimal[instrument] > 0,
                                                   trades_minimal[instrument] *
                                                   Instrument_BidPrice,
                                                   trades_minimal[instrument] * Instrument_AskPrice)

    if 'S' in instrument:

        Instrument_AskPrice = market_data[instrument, 'AskPrice']
        Instrument_BidPrice = market_data[instrument, 'BidPrice']

        valuation_dataframe[instrument] = np.where(trades_minimal[instrument] > 0,
                                                   trades_minimal[instrument] *
                                                   Instrument_BidPrice,
                                                   trades_minimal[instrument] * Instrument_AskPrice)

# Show DataFrame & Calculate total valuation
display(valuation_dataframe.tail())
total_valuation = valuation_dataframe[-1:].sum().sum()

print("Total valuation of our Position is currently: €", round(total_valuation, 2))

Stock    P60    P70     P80      C60      C70     C80
Timestamp                                                                    
2018-02-28 23:35:00 -76329.75  46.55  93.24 -3238.2  4971.50  4500.72  986.70
2018-02-28 23:40:00 -76253.80  45.60  91.80 -3234.0  4974.55  4516.68  975.26
2018-02-28 23:45:00 -76253.80  46.55  90.36 -3246.6  4962.35  4495.40  978.12
2018-02-28 23:50:00 -76253.80  43.70  90.00 -3234.0  4974.55  4532.64  989.56
2018-02-28 23:55:00 -76077.55  44.65  90.36 -3280.2  4944.05  4500.72  975.26

Total valuation of our Position is currently: € -68802.71


This code is creating a new dataframe that will be used to store information about trades. It first takes an existing dataframe, called "trades," and removes certain columns from it, using the "drop" function. It then creates a new dataframe, called "valuation_dataframe," which will be used to store information about market data. It uses a for loop to calculate valuations for certain instruments (instruments are basically financial products like stocks, options, etc.) The code then checks if the instrument name contains a certain letter, and based on that, it calculates the valuation and stores it in the valuation dataframe. Finally, the code displays the valuation and calculates the total valuation of the position by summing up the valuations for each instrument. The result is shown in euros.

In [18]:
# Create Empty DataFrame
blackscholes_dataframe = {}

# Create Columns based on Option Names
for option in option_names:

    if 'C' in option:
        blackscholes_dataframe[option] = []

    if 'P' in option:
        blackscholes_dataframe[option] = []

# Forloop that calculates the margins and thus profits
for time, data in market_data.iterrows():
    
    for option in option_names:

        if "C" in option:
            margin1 = data[option, 'BidPrice'] - data[option, 'Expected AskPrice']
            margin2 = data[option, 'Expected BidPrice'] - data[option, 'AskPrice']

            if margin1 > 0.10:
                blackscholes_dataframe[option].append(margin1)

            elif margin2 > 0.10:
                blackscholes_dataframe[option].append(margin2)

            else:
                blackscholes_dataframe[option].append(0)

        elif "P" in option:
            margin1 = data[option, 'BidPrice'] - data[option, 'Expected AskPrice']
            margin2 = data[option, 'Expected BidPrice'] - data[option, 'AskPrice']

            if margin1 > 0.10:
                blackscholes_dataframe[option].append(margin1)

            elif margin2 > 0.10:
                blackscholes_dataframe[option].append(margin2)

            else:
                blackscholes_dataframe[option].append(0)

# Create DataFrame with index of market_data
blackscholes_dataframe = pd.DataFrame(blackscholes_dataframe, index=market_data.index)

# Calculate Black_Scholes Profit
total_blackscholes = (abs(trades_diff).drop('Stock', axis=1) * blackscholes_dataframe).sum().sum()
print('The total profit from Black Scholes is: €',round(total_blackscholes,2))

The total profit from Black Scholes is: € 547.08


This code creates an empty table called "blackscholes_dataframe" and then fills it with values based on different options (buying and selling stocks at specific prices). It does this by using a loop to go through each option and calculate the profit for each one using market data (information about how the stock market is doing). It then creates a new table with the calculated profits and compares them to the actual amount of money made from trading stocks using the Black Scholes method (a mathematical formula for pricing options). Finally, it prints out the total profit made using Black Scholes.

In [19]:
# Blackscholes Dataframe times Volumes
blackscholes_dataframe = pd.DataFrame(
    abs(trades_diff).drop('Stock', axis=1) * blackscholes_dataframe)

# Cumulative Blackscholes
blackscholes_cumulative = {}

for column in blackscholes_dataframe.columns:

    blackscholes_cumulative[column] = blackscholes_dataframe[column].cumsum()

blackscholes_cumulative = pd.DataFrame(blackscholes_cumulative)

# Show Dataframe
blackscholes_cumulative.tail()

C60    C70    C80    P60     P70     P80
Timestamp                                                      
2018-02-28 23:35:00  61.95  77.29  88.98  14.33  135.36  169.17
2018-02-28 23:40:00  61.95  77.29  88.98  14.33  135.36  169.17
2018-02-28 23:45:00  61.95  77.29  88.98  14.33  135.36  169.17
2018-02-28 23:50:00  61.95  77.29  88.98  14.33  135.36  169.17
2018-02-28 23:55:00  61.95  77.29  88.98  14.33  135.36  169.17

This python code is taking a set of data and multiplying it by another set of data to get a new set of values. Then, it is taking each number in the new set and adding it to the previous number to get a cumulative total. Finally, it is organizing all of these numbers into a table to easily see and understand the data.

In [20]:
# Last PnL in the Table should match the below value
print('The total profit generated from the Option Arbitrage strategy is: €',
      round(total_cashflow + total_valuation + total_blackscholes, 2))

The total profit generated from the Option Arbitrage strategy is: € 729.91


It looks like this code calculates the total profit from an "Option Arbitrage strategy". The "total_cashflow", "total_valuation", and "total_blackscholes" are probably separate calculations that are added together to get the total profit. The "print()" function is used to display the result in a human-readable format, with the text "The total profit generated from the Option Arbitrage strategy is: €" followed by the total profit value. The "round()" function is used to round the total profit value to 2 decimal places to make it easier to read.